In [1]:
!pip install streamlit-chat
!pip install streamlit
!pip install langchain
!pip install faiss-cpu

In [2]:
import streamlit as st
from streamlit_chat import message
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationalRetrievalChain
from langchain.vectorstores import FAISS
import tempfile
from langchain.document_loaders import PyPDFLoader

import os
os.environ["OPENAI_API_KEY"] = "sk"
uploaded_file = st.sidebar.file_uploader("Upload", type="pdf")

2025-11-13 19:53:22.088 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-13 19:53:23.788 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-13 19:53:23.789 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-13 19:53:23.789 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-13 19:53:23.794 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-13 19:53:24.124 
  command:

    streamlit run C:\Users\chogu\anaconda3\envs\11m\lib\site-packages\ipy

In [3]:
if uploaded_file:
    with tempfile.NamedTemporaryFile(delete=False) as tmp_file:
        tmp_file.write(uploaded_file.getvalue())
        tmp_file_path = tmp_file.name

    loader = PyPDFLoader(tmp_file_path)
    data = loader.load()
    embeddings = OpenAIEmbeddings()
    vectors = FAISS.from_documents(data, embeddings)

    # ConversationalRetrievalChain으로 대화 형태 구성
    chain = ConversationalRetrievalChain.from_llm(
    llm=ChatOpenAI(temperature=0, model_name="gpt-4o-mini"),
    retriever=vectors.as_retriever())

    # 대화 저장을 위한 세션 state 구성(append)
    def conversational_chat(query):
        result = chain({"question": query, "chat_history": st.session_state["history"]})
        st.session_state["history"].append((query, result["answer"]))
        return result["answer"]

    if "history" not in st.session_state:
        st.session_state["history"] = []

    if "generated" not in st.session_state:
        st.session_state["generated"] = ["안녕하세요! " + uploaded_file.name + "에 관해 질문 주세요."]

    if "past" not in st.session_state:
        st.session_state["past"] = ["안녕하세요!"]

    # 컨테이너 생성
    response_container = st.container()
    container = st.container()

    with container:
        with st.form(key="my_form", clear_on_submit=True):
            user_input = st.text_input("Query:", placeholder="PDF 내용을 기반으로 질문하세요.", key="input")
            submit_button = st.form_submit_button(label="Send")

        if submit_button and user_input:
            output = conversational_chat(user_input)
            # 사용자의 입력과 LLM의 대답 기록을 저장 (append)
            st.session_state['past'].append(user_input)
            st.session_state['generated'].append(output)

    # LLM의 답변을 화면에 출력
    if st.session_state['generated']:
        with response_container:
            for i in range(len(st.session_state['generated'])):
                message(st.session_state['past'][i], is_user=True,
key=str(i) + '_user', avatar_style="fun-emoji", seed="Male")
                message(st.session_state['generated'][i], key=str(i),
avatar_style="bottts", seed="Fluffy")
